In [20]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms, models
import cv2
import time


In [26]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              #transforms.Lambda(lambda x: x.repeat(3, 1, 1) ),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
# Download and load the training data
trainSet = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=64, shuffle=True)

# Download and load the test data
testSet = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=64, shuffle=True)


In [27]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    

In [28]:
model = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
    nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)
epochs = 30
model.to(device);
#log softmax => more efficient than normal softmax

trainLosses, testLosses = [], []
for e in range(epochs):
    runningLoss = 0
    for images, labels in trainLoader:
        
        optimizer.zero_grad()
        images = images.view(images.shape[0], -1)
        modelLogOutput = model(images)
        loss = criterion(modelLogOutput, labels)
        #back propogation compute gradient of each parameter using the loss
        loss.backward()
        #Gradient Descent optimizes the parameters
        optimizer.step()
        
        runningLoss += loss.item()
        
    else:
        testLoss = 0
        accuracy = 0
        evaluationStart = time.time()
        
        # Turn off gradients for validation, reduces memory usage, and easier computations
        with torch.no_grad():
            model.eval()
            for images, labels in testLoader:
                images = images.view(images.shape[0], -1)
                modelLogOutput = model(images)
                testLoss += criterion(modelLogOutput, labels)
                
                #Converting the log output to a probility
                outputProbability = torch.exp(modelLogOutput)
                
                topProbability, predictedClass = outputProbability.topk(1, dim=1)
                predictedClass = torch.squeeze(predictedClass)
                equals = (predictedClass == labels)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        evaluationEnd = time.time()
        model.train()
        
        trainLosses.append(runningLoss/len(trainLoader))
        testLosses.append(testLoss/len(testLoader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(trainLosses[-1]),
              "Test Loss: {:.3f}.. ".format(testLosses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testLoader)),
              "Evaluation Time: {:.3f}".format((evaluationEnd-evaluationStart)),
              "Evaluation Time per Image: {:.3f}".format((evaluationEnd-evaluationStart)/len(testLoader)))

Epoch: 1/30..  Training Loss: 1.832..  Test Loss: 1.155..  Test Accuracy: 0.750 Evaluation Time: 1.764 Evaluation Time per Image: 0.011
Epoch: 2/30..  Training Loss: 0.806..  Test Loss: 0.577..  Test Accuracy: 0.849 Evaluation Time: 1.799 Evaluation Time per Image: 0.011
Epoch: 3/30..  Training Loss: 0.512..  Test Loss: 0.435..  Test Accuracy: 0.883 Evaluation Time: 1.783 Evaluation Time per Image: 0.011
Epoch: 4/30..  Training Loss: 0.420..  Test Loss: 0.382..  Test Accuracy: 0.893 Evaluation Time: 1.899 Evaluation Time per Image: 0.012
Epoch: 5/30..  Training Loss: 0.378..  Test Loss: 0.351..  Test Accuracy: 0.901 Evaluation Time: 1.778 Evaluation Time per Image: 0.011
Epoch: 6/30..  Training Loss: 0.353..  Test Loss: 0.331..  Test Accuracy: 0.907 Evaluation Time: 1.789 Evaluation Time per Image: 0.011
Epoch: 7/30..  Training Loss: 0.335..  Test Loss: 0.321..  Test Accuracy: 0.906 Evaluation Time: 1.741 Evaluation Time per Image: 0.011
Epoch: 8/30..  Training Loss: 0.322..  Test Loss